# `poke-env` Quickstart: Practical Examples and Snippets

Complete source code for this example is available [here](https://github.com/hsahovic/poke-env/blob/master/examples/cross_evaluate_random_players.ipynb).

**Note**: this notebooks requires a locally running Pokémon Showdown server. Please see the [getting started section](../getting_started.rst) for help on how to set one up.

## Creating Agents and Making Them Battle

### Creating Built-in Agents

`poke-env` comes with a few built-in agents. These agents are meant to be used as a baseline for your own agents.

The simplest agent is the `RandomPlayer` agent. This agent will select a random valid move at each turn. Let's create one:


In [1]:
from poke_env import RandomPlayer
from poke_env.data import GenData

# The RandomPlayer is a basic agent that makes decisions randomly,
# serving as a starting point for more complex agent development.
random_player = RandomPlayer()

### Creating a Battle

To create a battle, let's create a second agent and use the `battle_against` method. It's an asynchronous method, so we need to `await` it.

In [2]:
second_player = RandomPlayer()

# The battle_against method initiates a battle between two players.
# Here we are using asynchronous programming (await) to start the battle.
await random_player.battle_against(second_player, n_battles=1)

If you want to look at this battle, you can open a browser at [http://localhost:8000](http://localhost:8000) - you should see the battle in the lobby.

### Inspecting the Result

Here are a couple of ways to inspect the result of this battle.

In [3]:
# n_won_battles and n_finished_battles

print(f"Player {random_player.username} won {random_player.n_won_battles} out of {random_player.n_finished_battles} played")
print(f"Player {second_player.username} won {second_player.n_won_battles} out of {second_player.n_finished_battles} played")

# Looping over battles

for battle_tag, battle in random_player.battles.items():
    print(battle_tag, battle.won)

Player RandomPlayer 1 won 1 out of 1 played
Player RandomPlayer 2 won 0 out of 1 played
battle-gen9randombattle-169369 True


You can look at more properties of the [Player](../modules/player.rst) and [Battle](../modules/battle.rst) classes in the documentation.

### Running a Cross-Evaluation

`poke-env` provides a `cross_evaluate` function, that allows you to run a cross evaluation between multiple agents. It will run a number of battles between the two agents, and return the results of the evaluation in a structured way.

In [4]:
from poke_env import cross_evaluate

third_player = RandomPlayer()

players = [random_player, second_player, third_player]

cross_evaluation = await cross_evaluate(players, n_challenges=5)
cross_evaluation

{'RandomPlayer 1': {'RandomPlayer 1': None,
  'RandomPlayer 2': 0.6666666666666666,
  'RandomPlayer 3': 0.4},
 'RandomPlayer 2': {'RandomPlayer 1': 0.3333333333333333,
  'RandomPlayer 2': None,
  'RandomPlayer 3': 0.6},
 'RandomPlayer 3': {'RandomPlayer 1': 0.6,
  'RandomPlayer 2': 0.4,
  'RandomPlayer 3': None}}

Here's one way to pretty print the results of the cross evaluation using `tabulate`:

In [5]:
from tabulate import tabulate

table = [["-"] + [p.username for p in players]]
for p_1, results in cross_evaluation.items():
    table.append([p_1] + [cross_evaluation[p_1][p_2] for p_2 in results])

print(tabulate(table))

--------------  ------------------  ------------------  --------------
-               RandomPlayer 1      RandomPlayer 2      RandomPlayer 3
RandomPlayer 1                      0.6666666666666666  0.4
RandomPlayer 2  0.3333333333333333                      0.6
RandomPlayer 3  0.6                 0.4
--------------  ------------------  ------------------  --------------


## Building a Max Damage Player

In this section, we introduce the `MaxDamagePlayer`, a custom agent designed to choose moves that maximize damage output.

### Implementing the MaxDamagePlayer Class

The primary task is to override the choose_move method. This method, defined as `choose_move(self, battle: Battle) -> str`, requires a `Battle` object as input, representing the current game state, and outputs a move order as a string. This move order must adhere to the [showdown protocol](https://github.com/smogon/pokemon-showdown/blob/master/sim/SIM-PROTOCOL.md) format. The `poke-env` library provides the `create_order` method to assist in formatting move orders directly from `Pokemon` and `Move` objects.

The `battle` parameter, a `Battle` object, encapsulates the agent's current knowledge of the game state. It provides various properties for easy access to game details, such as `active_pokemon`, `available_moves`, `available_switches`, `opponent_active_pokemon`, `opponent_team`, and `team`.

For this example, we'll utilize `available_moves`, which gives us a list of `Move` objects available in the current turn.

Our focus in implementing `MaxDamagePlayer` involves two key steps: interpreting the game state information from the battle object and then generating and returning a correctly formatted move order.

In [6]:
from poke_env.player import Player

class MaxDamagePlayer(Player):
    def choose_move(self, battle):
        # Chooses a move with the highest base power when possible
        if battle.available_moves:
            # Iterating over available moves to find the one with the highest base power
            best_move = max(battle.available_moves, key=lambda move: move.base_power)
            # Creating an order for the selected move
            return self.create_order(best_move)
        else:
            # If no attacking move is available, perform a random switch
            # This involves choosing a random move, which could be a switch or another available action
            return self.choose_random_move(battle)


In the `choose_move` method, our first step is to determine if there are any available moves for the current turn, as indicated by `battle.available_moves`. When a move is available, we select the one with the highest `base_power`. Formatting our choice is achieved by the `create_order`.

However, there are scenarios where no moves are available. In such cases, we use `choose_random_move(battle)`. This method randomly selects either a move or a switch, and guarantees that we will return a valid order.

The `Player.create_order` function is a crucial part of this process. It's a wrapper method that generates valid battle messages. It can take either a `Move` or a `Pokemon` object as its input. When passing a `Move` object, additional parameters such as `mega`, `z_move`, `dynamax`, or `terastallize` can be specified to indicate special battle actions.

We will adjust our strategy to include `terastallize` at the earliest opportunity, enhancing the effectiveness of our player in battle scenarios.

In [7]:
class MaxDamagePlayer(Player):
    def choose_move(self, battle):
        if battle.available_moves:
            best_move = max(battle.available_moves, key=lambda move: move.base_power)
            
            if battle.can_tera:
                return self.create_order(best_move, terastallize=True)
            
            return self.create_order(best_move)
        else:
            return self.choose_random_move(battle)

### Testing the MaxDamagePlayer

Next, we'll test our `MaxDamagePlayer` against a `RandomPlayer` in a series of battles:


In [8]:
# Creating players
random_player = RandomPlayer()
max_damage_player = MaxDamagePlayer()

# Running battles
await max_damage_player.battle_against(random_player, n_battles=100)

# Displaying results
print(f"Max damage player won {max_damage_player.n_won_battles} / 100 battles")


Max damage player won 93 / 100 battles


Unsurprisingly, the `MaxDamagePlayer` wins most of the battles.

## Setting teams

Most formats do not provide a team automatically. 

To specify a team, you have two main options: you can either provide a str describing your team, or a `Teambuilder` object. This example will focus on the first option; if you want to learn more about using teambuilders, please refer to [Creating a custom teambuilder and Teambuilder: Parse, manage and generate showdown teams](custom_teambuilder.rst).

The easiest way to specify a team in poke-env is to copy-paste a showdown team. You can use showdown’s teambuilder and export it directly.

Alternatively, you can use showdown’s packed formats, which correspond to the actual string sent by the showdown client to the server.

Here is an example team, both in showdown and packed formats:

You can set a team by passing a `TeamBuilder` object or a `str` when initializing a `Player` object.

### Using a `str`

Here's an example:

In [9]:
team_1 = """
Goodra (M) @ Assault Vest
Ability: Sap Sipper
EVs: 248 HP / 252 SpA / 8 Spe
Modest Nature
IVs: 0 Atk
- Dragon Pulse
- Flamethrower
- Sludge Wave
- Thunderbolt

Sylveon (M) @ Leftovers
Ability: Pixilate
EVs: 248 HP / 244 Def / 16 SpD
Calm Nature
IVs: 0 Atk
- Hyper Voice
- Mystical Fire
- Protect
- Wish

Toxtricity (M) @ Throat Spray
Ability: Punk Rock
EVs: 4 Atk / 252 SpA / 252 Spe
Rash Nature
- Overdrive
- Boomburst
- Shift Gear
- Fire Punch

Seismitoad (M) @ Leftovers
Ability: Water Absorb
EVs: 252 HP / 252 Def / 4 SpD
Relaxed Nature
- Stealth Rock
- Scald
- Earthquake
- Toxic

Corviknight (M) @ Leftovers
Ability: Pressure
EVs: 248 HP / 80 SpD / 180 Spe
Impish Nature
- Defog
- Brave Bird
- Roost
- U-turn

Galvantula @ Focus Sash
Ability: Compound Eyes
EVs: 252 SpA / 4 SpD / 252 Spe
Timid Nature
IVs: 0 Atk
- Sticky Web
- Thunder Wave
- Thunder
- Energy Ball
"""
team_2 = """
Togekiss @ Leftovers
Ability: Serene Grace
EVs: 248 HP / 8 SpA / 252 Spe
Timid Nature
IVs: 0 Atk
- Air Slash
- Nasty Plot
- Substitute
- Thunder Wave

Galvantula @ Focus Sash
Ability: Compound Eyes
EVs: 252 SpA / 4 SpD / 252 Spe
Timid Nature
IVs: 0 Atk
- Sticky Web
- Thunder Wave
- Thunder
- Energy Ball

Cloyster @ Leftovers
Ability: Skill Link
EVs: 252 Atk / 4 SpD / 252 Spe
Adamant Nature
- Icicle Spear
- Rock Blast
- Ice Shard
- Shell Smash

Sandaconda @ Focus Sash
Ability: Sand Spit
EVs: 252 Atk / 4 SpD / 252 Spe
Jolly Nature
- Stealth Rock
- Glare
- Earthquake
- Rock Tomb

Excadrill @ Focus Sash
Ability: Sand Rush
EVs: 252 Atk / 4 SpD / 252 Spe
Adamant Nature
- Iron Head
- Rock Slide
- Earthquake
- Rapid Spin

Cinccino @ Leftovers
Ability: Skill Link
EVs: 252 Atk / 4 Def / 252 Spe
Jolly Nature
- Bullet Seed
- Knock Off
- Rock Blast
- Tail Slap
"""

p1 = MaxDamagePlayer(battle_format="gen8ou", team=team_1)
p2 = MaxDamagePlayer(battle_format="gen8ou", team=team_2)

await p1.battle_against(p2, n_battles=1)

### Dealing with team preview

By default, teampreview will be handled randomly. You can override this behaviour by overriding the `teampreview` method of the `Player` class. Here is a nexample using type-based heuristics:

In [10]:
import numpy as np

def teampreview_performance(mon_a, mon_b):
    # We evaluate the performance on mon_a against mon_b as its type advantage
    a_on_b = b_on_a = -np.inf
    for type_ in mon_a.types:
        if type_:
            a_on_b = max(a_on_b, type_.damage_multiplier(*mon_b.types, type_chart=GenData.from_gen(8).type_chart))
    # We do the same for mon_b over mon_a
    for type_ in mon_b.types:
        if type_:
            b_on_a = max(b_on_a, type_.damage_multiplier(*mon_a.types, type_chart=GenData.from_gen(8).type_chart))
    # Our performance metric is the different between the two
    return a_on_b - b_on_a

class MaxDamagePlayerWithTeampreview(MaxDamagePlayer):
    def teampreview(self, battle):
        mon_performance = {}

        # For each of our pokemons
        for i, mon in enumerate(battle.team.values()):
            # We store their average performance against the opponent team
            mon_performance[i] = np.mean(
                [
                    teampreview_performance(mon, opp)
                    for opp in battle.opponent_team.values()
                ]
            )

        # We sort our mons by performance
        ordered_mons = sorted(mon_performance, key=lambda k: -mon_performance[k])

        # We start with the one we consider best overall
        # We use i + 1 as python indexes start from 0
        #  but showdown's indexes start from 1
        return "/team " + "".join([str(i + 1) for i in ordered_mons])
    
p3 = MaxDamagePlayerWithTeampreview(battle_format="gen8ou", team=team_1)
p4 = MaxDamagePlayerWithTeampreview(battle_format="gen8ou", team=team_2)

await p3.battle_against(p4, n_battles=1)

## Other Initialization Options for `Player` Objects

### Specifying an Avatar

You can specify an `avatar` argument when initializing a `Player` object. This argument is a string, corresponding to the avatar's name.

You can find a [list of avatar names here](https://github.com/smogon/pokemon-showdown-client/blob/6d55434cb85e7bbe614caadada819238190214f6/play.pokemonshowdown.com/src/battle-dex-data.ts#L690). If the avatar you are looking for is not in this list, you can inspect the message the client is sending to the server by opening your browser's development console and selecting the avatar manually.


In [11]:
player_with_avatar = RandomPlayer(avatar="boarder")

### Saving Battle Replays

You can save battle replays by specifying a `save_replay` value when initializing a `Player` object. This argument can either be a boolean (if `True`, the replays will be saved in the `replays`) or a string - in which case the replays will be saved in the specified directory.

In [12]:
player_with_replays = RandomPlayer(save_replays="my_folder")

### Logging

Every `Player` instance has a custom logger. By default, it will only surface warnings and errors. You can change the logging level by specifying a `log_level` argument when initializing a `Player` object.

The two most relevant values are `logging.INFO` or 20, which will surface every message sent or received by the client (which is very useful when debugging) and 25, which is a custom level used by `poke-env` to surface only the most relevant events.

You can also use `logging.DEBUG` or 10, but the difference with `logging.INFO` should only be relevant for `poke-env` internals.

In [13]:
verbose_player = RandomPlayer(log_level=20)

from asyncio import sleep
await sleep(1)

2023-12-16 03:09:18,500 - RandomPlayer 7 - INFO - Starting listening to showdown websocket
2023-12-16 03:09:18,504 - RandomPlayer 7 - INFO - <<< |updateuser| Guest 148|0|169|{"blockChallenges":false,"blockPMs":false,"ignoreTickets":false,"hideBattlesFromTrainerCard":false,"blockInvites":false,"doNotDisturb":false,"blockFriendRequests":false,"allowFriendNotifications":false,"displayBattlesToFriends":false,"hideLogins":false,"hiddenNextBattle":false,"inviteOnlyNextBattle":false,"language":null}
|customgroups|[{"symbol":"&","name":"Administrator","type":"leadership"},{"symbol":"#","name":"Room Owner","type":"leadership"},{"symbol":"★","name":"Host","type":"leadership"},{"symbol":"@","name":"Moderator","type":"staff"},{"symbol":"%","name":"Driver","type":"staff"},{"symbol":"§","name":"Section Leader","type":"staff"},{"symbol":"*","name":"Bot","type":"normal"},{"symbol":"☆","name":"Player","type":"normal"},{"symbol":"+","name":"Voice","type":"normal"},{"symbol":"^","name":"Prize Winner","ty

### Concurrency

By default, a `poke-env` `Player` will only run a single battle at a time. You can change this behavior by specifying a `max_concurrent_battles` argument when initializing a `Player` object.

This argument is an integer, and represents the maximum number of battles a `Player` can run at the same time. If 0, no limit will be enforced.

This can provide a significant speedup when your process is not CPU bound.

In [14]:
import time

# Time to run 50 battles, one at a time
start = time.time()
await random_player.battle_against(second_player, n_battles=50)
end = time.time()
print(f"Time to run 50 battles, one at a time: {end - start:.2f} seconds")

Time to run 50 battles, one at a time: 18.87 seconds


In [15]:
unrestricted_random_player = RandomPlayer(max_concurrent_battles=0)
unrestricted_second_player = RandomPlayer(max_concurrent_battles=0)

# Time to run 50 battles, in parallel
start = time.time()
await unrestricted_random_player.battle_against(unrestricted_second_player, n_battles=50)
end = time.time()
print(f"Time to run 50 battles, in parallel: {end - start:.2f} seconds")

Time to run 50 battles, in parallel: 4.54 seconds


Other options can also be used on the server side to make battles run faster.

### Pokemon Showdown Timer

You can turn on the Pokemon Showdown timer by setting `start_timer_on_battle_start` to `True` when initializing a `Player` object.

This is mostly relevant when pitting your argents against humans.

In [16]:
impatient_player = RandomPlayer(start_timer_on_battle_start=True)